<a href="https://colab.research.google.com/github/sunnydada6/django_project/blob/main/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd 

In [3]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git 

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [4]:
df=pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx')

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report 

In [6]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-ojt7nyne
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-ojt7nyne
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=11756 sha256=488e2b243e359baf985c90cb41e24a35785aab52bca4ca0645228faaf681d0d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-tib661rd/wheels/0d/b3/29/bfe3deffda68980088d17b81331be6667e837ffb4a071bae82
Successfully built preprocess-kgptalkie


In [7]:
import preprocess_kgptalkie as ps
import re 
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [8]:
df['Reviews']=df['Reviews'].apply(lambda x: get_clean(x)) 

In [9]:
df.head() 

,Reviews,Sentiment
0,when i first tuned in on this morning news i t...,neg
1,mere thoughts of going overboard aka babes aho...,neg
2,why does this movie fall well below standards ...,neg
3,wow and i thought that any steven segal movie ...,neg
4,the story is seen before but that doesand matt...,neg


In [10]:
tfidf=TfidfVectorizer(max_features=5000)
x=df['Reviews']
y=df['Sentiment']  
x=tfidf.fit_transform(x)




In [11]:

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=0)

In [12]:
clf=LinearSVC()
clf.fit(x_train,y_train)


LinearSVC()

In [13]:
x

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2845641 stored elements in Compressed Sparse Row format>

In [17]:
y_pred=clf.predict(x_test)

In [20]:
print(classification_report(y_test,y_pred))  

              precision    recall  f1-score   support

         neg       0.87      0.87      0.87      2480
         pos       0.87      0.88      0.88      2520

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [21]:
x='this movie is amazing. but some scenes are disspointed' 
x=get_clean(x)
vect=tfidf.transform([x])

In [23]:
vect.shape

(1, 5000)

In [25]:
clf.predict(vect)

array(['pos'], dtype=object)

In [26]:
import pickle 
pickle.dump(clf,open('model','wb'))